<a href="https://colab.research.google.com/github/chu-ise/411A-2022/blob/main/notebooks/02/01_hydra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%%capture
%pip install gdown hydra-core

In [10]:
import gdown, os
id = "13_3pQBnPtF4ASy5vFouh67HKLoEgJSdB"

data_file = "cloud_app.zip"
_dir = data_file.split(".")[0]
if not os.path.exists(_dir):
    gdown.download(id=id, output=data_file, quiet=False)
    !unzip $data_file
    os.remove(data_file)
else:
    print(f"{_dir} already exists")

Downloading...
From: https://drive.google.com/uc?id=13_3pQBnPtF4ASy5vFouh67HKLoEgJSdB
To: /content/cloud_app.zip
100%|██████████| 3.27k/3.27k [00:00<00:00, 2.87MB/s]

Archive:  cloud_app.zip
   creating: cloud_app/
   creating: cloud_app/conf/
  inflating: cloud_app/conf/__init__.py  
   creating: cloud_app/conf/application/
  inflating: cloud_app/conf/application/bananas.yaml  
  inflating: cloud_app/conf/application/donkey.yaml  
   creating: cloud_app/conf/cloud_provider/
  inflating: cloud_app/conf/cloud_provider/aws.yaml  
  inflating: cloud_app/conf/cloud_provider/local.yaml  
  inflating: cloud_app/conf/config.yaml  
   creating: cloud_app/conf/db/
  inflating: cloud_app/conf/db/mysql.yaml  
  inflating: cloud_app/conf/db/sqlite.yaml  
   creating: cloud_app/conf/environment/
  inflating: cloud_app/conf/environment/production.yaml  
  inflating: cloud_app/conf/environment/testing.yaml  


In [11]:
import os
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf

# Initializing Hydra
There are several ways to initialize. See the [API docs](https://hydra.cc/docs/next/advanced/compose_api/#api-documentation) for full details.
All methods support both a function call style that changes the global state, and a context style that cleans up when the scope exits.

## initialize()
Initializes Hydra and add the config_path to the config search path.
config_path is relative to the parent of the caller, in this case it is realtive to the directory containing
this Notebook.

In [12]:
with initialize(config_path="cloud_app/conf"):
    cfg = compose(overrides=["+db=mysql"])
    print(cfg)

{'db': {'driver': 'mysql', 'user': '???', 'pass': '???'}}


## initialize_config_module()
Initializes Hydra and add the config_module to the config search path.  
The config module must be importable (an `__init__.py` must exist at its top level)

In [13]:
with initialize_config_module(config_module="cloud_app.conf"):
    cfg = compose(overrides=["+db=mysql"])
    print(cfg)

{'db': {'driver': 'mysql', 'user': '???', 'pass': '???'}}


## initialize_config_dir()
Initializes Hydra and add the config_path to the config search path.
The config_path must be an absolute path on the file system.

In [14]:
abs_config_dir=os.path.abspath("cloud_app/conf")
with initialize_config_dir(config_dir=abs_config_dir):
    cfg = compose(overrides=["+db=mysql"])
    print(cfg)

{'db': {'driver': 'mysql', 'user': '???', 'pass': '???'}}


## Global initialization
Calling each of the initilizaiton methods outside of a context changes the global state.

In [15]:
initialize(config_path="cloud_app/conf")
compose(overrides=["+db=mysql"])

{'db': {'driver': 'mysql', 'user': '???', 'pass': '???'}}

# Config composition
Below are some more interesting demonstrations of config composition
## Config Structure
The `__init__.py` file is needed is needed to help Python find the config files in some scenarios (It can be empty).  
```
conf/
├── application
│   ├── bananas.yaml
│   └── donkey.yaml
├── cloud_provider
│   ├── aws.yaml
│   └── local.yaml
├── db
│   ├── mysql.yaml
│   └── sqlite.yaml
├── environment
│   ├── production.yaml
│   └── testing.yaml
├── config.yaml
└── __init__.py
```

In [16]:
# compose application specific config (in this case the applicaiton is "donkey")
cfg=compose(overrides= ["+db=mysql", "+environment=production", "+application=donkey"])
print(OmegaConf.to_yaml(cfg))

db:
  driver: mysql
  user: mysql
  pass: r4Zn*jQ9JB1Rz2kfz
donkey:
  name: kong
  rank: king



In [17]:
# compose from config.yaml, this composes a bunch of defaults in:
cfg=compose(config_name="config.yaml")
print(OmegaConf.to_yaml(cfg))

db:
  driver: sqlite
  user: test
  pass: test
  file: test.db
cloud:
  name: local
  host: localhost
  port: 9876



In [18]:
import os
os.environ["AWS_API_KEY"] = "__SOMETHING_FROM_AN_ENV_VARIABLE__"

# compose from config.yaml and override from the default testing
# environment to production and cloud provider to aws.
# Also override the ami id

cfg=compose(
    config_name="config.yaml", 
    overrides=[
        "cloud_provider=aws",
        "environment=production",
        "cloud.ami_id=MY_AMI_ID",
    ]
)
print(OmegaConf.to_yaml(cfg, resolve=True))

db:
  driver: sqlite
  user: mysql
  pass: r4Zn*jQ9JB1Rz2kfz
  file: test.db
cloud:
  name: aws
  api_key: __SOMETHING_FROM_AN_ENV_VARIABLE__
  ami_id: MY_AMI_ID

